In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [17]:
path = "C:\\Users\\kklausmeyer\\Documents\\TNC_Projects\\Freshwater\\SB88"
cuahsi_path = path + "\\Staten\\data\\CUAHSI"
acc_path = path + "\\Staten\\data\\Accuracy_Assessment"
version = "20180403"
in_xls = cuahsi_path + "\\StatenX_Tableau_" + version + ".xlsx"
sheet_name = "All_methods"

hourly_data = acc_path + "\\hourly_data.csv"
pct_hourly_csv = acc_path + "\\pct_error_hourly_sum.csv"
count_hourly_csv = acc_path + "\\count_hourly_sum.csv"
pct_15min_csv = acc_path + "\\pct_error_15min_sum.csv"
count_15min_csv = acc_path + "\\count_15min_sum.csv"

In [20]:
#Import the data from excel format.  Takes about 1 min
xl = pd.ExcelFile(in_xls)
df = xl.parse(sheet_name)
df.head()

,DataValue,LocalDateTime,UTCOffset,DateTimeUTC,SiteCode,VariableCode,QualifierCode,MethodCode,SourceCode,QualityControlLevelCode,CensorCode,Unnamed: 11
0,0.0,2017-06-29 15:44:50,-7,2017-06-29 22:44:50,Siphon7,GPM_2min,0,AG3000_DL,TNC,0,nc,7.0
1,0.0,2017-06-29 15:46:50,-7,2017-06-29 22:46:50,Siphon7,GPM_2min,0,AG3000_DL,TNC,0,nc,7.0
2,0.0,2017-06-29 15:48:50,-7,2017-06-29 22:48:50,Siphon7,GPM_2min,0,AG3000_DL,TNC,0,nc,7.0
3,0.0,2017-06-29 15:50:50,-7,2017-06-29 22:50:50,Siphon7,GPM_2min,0,AG3000_DL,TNC,0,nc,7.0
4,0.0,2017-06-29 15:52:50,-7,2017-06-29 22:52:50,Siphon7,GPM_2min,0,AG3000_DL,TNC,0,nc,7.0


In [21]:
df_archive = df

In [24]:
# Drop no data values (-9999)

df = df.loc[df['DataValue'] != -9999]

In [26]:
# Summarize by hour and by month
df['hour'] = df['DateTimeUTC'].dt.round('60min')
df['month'] = df['DateTimeUTC'].dt.month

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
list(df)

['DataValue',
 'LocalDateTime',
 'UTCOffset',
 'DateTimeUTC',
 'SiteCode',
 'VariableCode',
 'QualifierCode',
 'MethodCode',
 'SourceCode',
 'QualityControlLevelCode',
 'CensorCode',
 'Unnamed: 11',
 'hour',
 'month']

In [44]:
# Get average values for each hour

group_list = ['hour','month','MethodCode','QualifierCode','SiteCode','VariableCode','QualityControlLevelCode']

df_hour = (df.groupby(group_list)
           .agg({'DataValue':'mean'})
           .reset_index()
           .rename(columns={'DataValue':'Mean_DataValue'})
          )

In [45]:
# Get the AG3000 data for testing

df_control = df_hour.loc[df_hour['MethodCode'] == "AG3000_DL"]

In [46]:
df_control.rename(columns={'Mean_DataValue': 'Control_GPM'}, inplace=True)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [47]:
df_control = df_control.drop(['month','MethodCode','SiteCode','QualifierCode','VariableCode','QualityControlLevelCode'], axis=1)

In [48]:
df_control

,hour,Control_GPM
1271,2017-06-29 23:00:00,1380.633478
1274,2017-06-30 00:00:00,784.288667
1277,2017-06-30 01:00:00,0.000000
1280,2017-06-30 02:00:00,0.000000
1283,2017-06-30 03:00:00,0.000000
1286,2017-06-30 04:00:00,0.000000
1289,2017-06-30 05:00:00,0.000000
1292,2017-06-30 06:00:00,0.000000
1295,2017-06-30 07:00:00,0.000000
1298,2017-06-30 08:00:00,0.000000


In [49]:
# Join the control data to the testing data
df_hour = pd.merge(df_hour, df_control, on='hour', how='outer')

In [50]:
# Make a new column for absolute difference

df_hour["ABS"] = df_hour["Mean_DataValue"].subtract(df_hour["Control_GPM"], axis=0).abs()

In [51]:
# Count records with measurements
pd.DataFrame(df_hour.groupby(by=["MethodCode", "QualifierCode"])["ABS"].count().reset_index())

,MethodCode,QualifierCode,ABS
0,AG3000_DL,0,5231
1,AG3000_EM,0,1402
2,AG3000_EM,1,1
3,EX250_WE,1,31
4,EX250_WE,2,5230
5,IS203_EM,0,1832
6,IS203_EM,1,150
7,IS203_EM,2,193
8,IS207i_WE,0,929
9,IS207i_WE,1,0


In [52]:
# Make a new column for percent difference

df_hour["PCT_diff"] = df_hour["ABS"].divide(df_hour["Control_GPM"], axis=0)

In [53]:
# Convert infinity values to 1
df_hour = df_hour.replace([np.inf, -np.inf], 1)

In [54]:
# Export hourly data to csv
df_hour.to_csv(hourly_data)

In [55]:
df_hour

,hour,month,MethodCode,QualifierCode,SiteCode,VariableCode,QualityControlLevelCode,Mean_DataValue,Control_GPM,ABS,PCT_diff
0,2017-06-01 07:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN
1,2017-06-01 08:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN
2,2017-06-01 09:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN
3,2017-06-01 10:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN
4,2017-06-01 11:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN
5,2017-06-01 12:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN
6,2017-06-01 13:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN
7,2017-06-01 14:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN
8,2017-06-01 15:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN
9,2017-06-01 16:00:00,6,EX250_WE,2,Siphon7,GPM_15min,0,0.000000,NaN,NaN,NaN


In [40]:
# Drop zero values from measurement
df_hour = df_hour.loc[df_hour['Mean_DataValue'] > 0]

In [42]:
# Calculate average percent error
pct_hour = pd.DataFrame(df_hour.groupby(by=["MethodCode","QualifierCode"])["PCT_diff"].mean().reset_index())
pct_hour.to_csv(pct_hourly_csv)
pct_hour

,MethodCode,QualifierCode,PCT_diff
0,AG3000_DL,0,0.000000
1,AG3000_EM,0,0.014774
2,EX250_WE,1,1.115289
3,EX250_WE,2,0.166838
4,IS203_EM,0,0.025750
5,IS203_EM,1,0.889347
6,IS203_EM,2,0.141278
7,IS207i_WE,0,0.023124
8,ORCAS_AY,0,0.014235


In [43]:
# Count records with measurements
count_hour = pd.DataFrame(df_hour.groupby(by=["MethodCode", "QualifierCode"])["ABS"].count().reset_index())
count_hour.to_csv(count_hourly_csv)
count_hour

,MethodCode,QualifierCode,ABS
0,AG3000_DL,0,787
1,AG3000_EM,0,108
2,EX250_WE,1,31
3,EX250_WE,2,792
4,IS203_EM,0,142
5,IS203_EM,1,17
6,IS203_EM,2,72
7,IS207i_WE,0,240
8,ORCAS_AY,0,95
